In [2]:
import numpy as np
import pandas as pd
from siuba import *
from calitp import *
import intake
import shared_utils

import data_prep
import _utils

In [3]:
df = pd.read_parquet('BC_GTFS_NTD.parquet')

In [6]:
df.sample()

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,...,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,itp_id,Is_Agency_In_BC_Only_1_means_Yes,GTFS,fleet_size
173,2017,Section 5311(f),CA-2017-168 | 0017000259-F,2017,Monterey-Salinas Transit,BCG0000279,Construction - Bus Shelters,119302,64CC17-00452,5685.0,...,65.0,0.0,186.0,0.0,65.0,251.0,208.0,0,Research,Large


In [10]:
df>>filter(_.organization_name=="Madera County")>>count(_.ali)

,ali,n
0,111215,2
1,111304,5
2,117A00,5
3,300902,6


In [16]:
df>>group_by(_.organization_name)>>summarize(n = _.activebalance.mean())>>filter(_.n<=0)

,organization_name,n
5,City of Arcata,0.000000
14,"City of Eureka, dba: Eureka Transit Service",0.000000
15,City of Fairfield,0.000000
44,"Greyhound Lines, Inc.",-39110.769000
48,Kings County Area Public Transit Agency,-3894.727273


In [20]:
column_names = ['_16_20', '_21_25', '_26_30', '_31_60']
df['_16plus']= df[column_names].sum(axis=1)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 798 entries, 0 to 797
Data columns (total 45 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   grant_fiscal_year                   798 non-null    int64  
 1   funding_program                     798 non-null    object 
 2   grant_number                        798 non-null    object 
 3   project_year                        798 non-null    int64  
 4   organization_name                   798 non-null    object 
 5   upin                                798 non-null    object 
 6   description                         798 non-null    object 
 7   ali                                 798 non-null    object 
 8   contract_number                     682 non-null    object 
 9   allocationamount                    798 non-null    float64
 10  encumbered_amount                   798 non-null    float64
 11  expendedamount                      798 non-n

In [22]:
df_agg = df.groupby(['organization_name',
                     'reporter_type',
                     'GTFS',
                     'fleet_size',
                     'ntd_id',
                     'itp_id']).agg({'allocationamount':'mean',
                                         'encumbered_amount':'mean',
                                         'expendedamount':'mean',
                                         'encumbered_amount':'mean',
                                         'activebalance':'mean',
                                         'closedoutbalance':'mean',
                                         'expendedamount':'mean',
                                         'total_vehicles':'max',
                                         'average_age_of_fleet__in_years_':'max',
                                         'average_lifetime_miles_per_vehicle':'max',
                                         'Automobiles':'max',
                                         'Bus':'max',
                                         'Other':'max',
                                         'Train':'max',
                                         'Van':'max',
                                         'automobiles_door':'max',
                                         'bus_doors':'max',
                                         'van_doors':'max',
                                         'train_doors':'max',
                                         'doors_sum':'max',
                                         '_0_9':'max',
                                         '_10_12':'max',
                                         '_13_15':'max',
                                         '_16_20':'max',
                                         '_21_25':'max',
                                         '_26_30':'max',
                                         '_31_60':'max',
                                         '_16plus':'max',
                                         '_60plus':'max'})


df_agg= df_agg.reset_index()

df_agg.head()


,organization_name,reporter_type,GTFS,fleet_size,ntd_id,itp_id,allocationamount,encumbered_amount,expendedamount,activebalance,...,doors_sum,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_16plus,_60plus
0,Amador Regional Transit System,Rural Reporter,Ok,Small,9R02-91000,11.0,189817.375000,159889.250000,133498.000000,56319.375000,...,21.0,18.0,3.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0
1,Butte County Association of Governments,Full Reporter,Ok,Medium,90208,47.0,490548.916667,401737.500000,363577.065000,126971.851667,...,82.0,44.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Calaveras Transit Agency,Rural Reporter,None,Small,9R02-99442,0.0,187646.000000,212771.888889,120177.108889,67468.891111,...,11.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Central Contra Costa Transit Authority, dba: C...",Full Reporter,None,Large,90078,0.0,50412.000000,50412.000000,0.000000,50412.000000,...,312.0,160.0,43.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0
4,City of Arcata,Rural Reporter,Ok,Small,9R02-91018,18.0,82111.428571,143540.000000,82111.428571,0.000000,...,12.0,5.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
#df_agg.to_parquet("5311_agg.parquet")